### MindSpore的多卡并行设计

不同于Pytorch提供DataParallel的单进程多卡训练，MindSpore采用多进程并行的方式。其优势是不存在所有梯度需要汇集到0号卡而导致的多GPU显存占用不均匀问题。且单卡和多卡的训练代码不需要做太大的改动，仅需要通过数行代码配置启动即可。

但此类设计需要进程间通信，且需要DataLoader(即mindspore.dataset)的支持。下面通过几个例子进行演示：

#### AllReduce
将所有向量相加，并将结果广播给所有GPU

In [1]:
!mpirun --allow-run-as-root -np 2 python allreduce.py

allreduce之前：[[2. 2.]] at rank 1
allreduce之前：[[1. 1.]] at rank 0
allreduce之后：[[3. 3.]] at rank 1
allreduce之后：[[3. 3.]] at rank 0


#### AlltoAll
将一个小批量数据均匀地分布在多个GPU上

In [2]:
!mpirun --allow-run-as-root -np 2 python alltoall.py

alltoall之前：[0 1 2 3 4 5 6 7] at rank 0
alltoall之前：[0 1 2 3 4 5 6 7] at rank 1
alltoall之后：[4 5 6 7] at rank 1
alltoall之后：[0 1 2 3] at rank 0
